# Nettoyage des données

Après avoir scrapé, il nous faut tenir compte de plusieurs choses pour nettoyer les bases de données et les fusionner :
- dans un premier temps, on convertit les dates dans un format date permettant ensuite de trier et de réarranger la table dans le bon ordre temporel.
- les cases vides susceptibles d'apparaître dans les colonnes "Likes", "Views", etc... sont remplacées par des 0 (il n'y a eu en effet aucun like par exemple).
- on fusionne les tables et on les trie par date
- enfin, on supprime les bots, définis dans notre code par une répétition de 4 tweets faisant moins de 1000 vues (car certains tweets qui se répètent peuvent être des titres d'article)


In [1]:
!pip install pandas 
!pip install datetime
!pip install openpyxl

## 1. Convertir les dates en format date

Dans le code suivant, nous cherchons d'une part à remplacer les 2.3K par 2300, à convertir les dates dans un format *datetime*.

Nous avons fait le choix d'ajouter deux colonnes :
- une colonne YearWeek sous format 2024-1 pour la première semaine de 2024
- une colonne YearMonth sous format 2024-10 pour le mois d'octobre 2024

Nous avons décidé de nous restreindre à l'année 2024, en excluant le mois de décembre (car pas complet) : en effet, les fluctuations de 2023 étaient relativement similaires à celles de début de début 2024.

In [2]:
import pandas as pd
import os
from datetime import datetime, timedelta

In [6]:
# Fonctions auxiliaires
def convertir_en_nombre(valeur):
    if pd.isna(valeur):
        return 0
    if isinstance(valeur, str):
        valeur = valeur.strip()
        if 'K' in valeur:
            return float(valeur.replace('K', '')) * 1000
        elif 'M' in valeur:
            return float(valeur.replace('M', '')) * 1000000
    try:
        return float(valeur)
    except ValueError:
        return 0

def convert_date(value, previous_date):
    try:
        value = str(value).strip()
        if len(value.split()) == 2:
            value_with_year = value + " 2024"
            return datetime.strptime(value_with_year, '%b %d %Y')
        elif len(value.split()) == 3:
            return datetime.strptime(value, '%b %d, %Y')
        else:
            return pd.NaT
    except Exception as e:
        print(f"Erreur lors de la conversion de la date : {value} -> {e}")
        return pd.NaT

def process_file(file_name, input_folder, output_folder):
    file_path = os.path.join(input_folder, file_name)
    df = pd.read_excel(file_path)
    
    colonnes_a_nettoyer = ['Comments', 'Repost', 'Likes', 'Views']
    for colonne in colonnes_a_nettoyer:
        if colonne in df.columns:
            df[colonne] = df[colonne].apply(convertir_en_nombre)
    
    converted_dates = []
    previous_date = None
    for value in df['Date']:
        if previous_date is None:
            converted_date = convert_date(value, datetime(2024, 1, 1))
        else:
            converted_date = convert_date(value, previous_date)
        converted_dates.append(converted_date)
        if pd.notna(converted_date):
            previous_date = converted_date
    
    df['ConvertedDate'] = converted_dates
    df['YearWeek'] = df['ConvertedDate'].dt.strftime('%Y-%U')
    df['YearMonth'] = df['ConvertedDate'].dt.strftime('%Y-%m')
    
    start_date = datetime(2024, 1, 1)
    end_date = datetime(2024, 11, 30)
    df_filtered = df[(df['ConvertedDate'] >= start_date) & (df['ConvertedDate'] <= end_date)]
    
    output_file = os.path.join(output_folder, file_name.replace('.xlsx', '_date.xlsx'))
    df_filtered.to_excel(output_file, index=False)

# Configuration des dossiers
notebook_dir = os.getcwd()
input_folder = os.path.join(notebook_dir, '../data_processing')
output_folder = os.path.join(notebook_dir, '../data_processing')
os.makedirs(output_folder, exist_ok=True)

# Liste des fichiers à traiter
files = [
    'tweets_stay_1.xlsx', 'tweets_exode_1.xlsx', 'tweets_exode_2.xlsx', 
    'tweets_last_2.xlsx', 'tweets_last_3.xlsx', 'tweets_last_4.xlsx', 
    'tweets_last_5.xlsx', 'tweets_last_6.xlsx', 'tweets_last.xlsx', 
    'tweets_leave_p1.xlsx', 'tweets_quit.xlsx'
]

# Traiter les fichiers
for file_name in files:
    process_file(file_name, input_folder, output_folder)


## 2. Fusion des tables et tri par date

On fusionne les tables et on trie par date : on obtient finalement la base **tweets_fusionnes.xlsx**.

In [8]:
input_folder = os.path.join(notebook_dir, '../data_processing')  # Dossier contenant les fichiers nettoyés

# Liste des fichiers à charger (chemins relatifs au dossier "tweets_nettoye")
file_list = [
    'tweets_stay_1_date.xlsx', 'tweets_exode_1_date.xlsx', 'tweets_exode_2_date.xlsx', 
    'tweets_last_2_date.xlsx', 'tweets_last_3_date.xlsx', 'tweets_last_4_date.xlsx', 
    'tweets_last_5_date.xlsx', 'tweets_last_6_date.xlsx', 'tweets_last_date.xlsx', 
    'tweets_leave_p1_date.xlsx', 'tweets_quit_date.xlsx'
]

# Charger et fusionner tous les fichiers
dataframes = []  # Liste pour stocker les DataFrames
for file_name in file_list:
    # Construire le chemin complet pour chaque fichier
    file_path = os.path.join(input_folder, file_name)
    
    # Charger chaque fichier
    df = pd.read_excel(file_path)
    dataframes.append(df)  

# Fusionner tous les DataFrames en un seul
merged_df = pd.concat(dataframes, ignore_index=True)


## 3. Suppression des bots

Nous nous sommes rendus compte que certains tweets revenaient plusieurs fois, après avoir réfléchi et testé plusieurs façons de supprimer les bots, nous nous sommes arrêtés à la définition d'un bot comme un tweet identique qui revient 4 fois ou plus, et qui fait moins de 1000 vues. 

In [ ]:
def clean_tweets(df, output_cleaned):
    """
    Nettoie les tweets en supprimant les tweets répétitifs avec moins de 1000 vues
    dans leur groupe et sauvegarde les résultats.
    """
    # Compter les occurrences de chaque tweet
    content_counts = df['Content'].value_counts()
    print(f"Total de contenus uniques : {len(content_counts)}")

    # Filtrer les contenus apparaissant au moins 4 fois
    repeated_contents = content_counts[content_counts >= 4].index
    print(f"Nombre de contenus répétés (>= 4 fois) : {len(repeated_contents)}")

    # Identifier les tweets répétitifs
    repeated_tweets = df[df['Content'].isin(repeated_contents)]

    # Identifier les groupes où un tweet a plus de 1000 vues
    to_keep = repeated_tweets.groupby('Content')['Views'].max()

    valid_tweets = to_keep[to_keep > 1000].index

    # Tweets à supprimer
    tweets_to_remove = repeated_tweets[~repeated_tweets['Content'].isin(valid_tweets)]
    print(f"Nombre de tweets à supprimer : {len(tweets_to_remove)}")

    # Retirer explicitement les tweets à supprimer de la base originale
    df_cleaned = df[~df.index.isin(tweets_to_remove.index)]

    # Trier les données nettoyées
    df_cleaned = df_cleaned.sort_values(by='ConvertedDate', ascending=False)

    # Sauvegarder la base nettoyée
    df_cleaned.to_excel(output_cleaned, index=False)

    print(f"{len(tweets_to_remove)} tweets supprimés")


# Définir les répertoires source et destination
data_processing_dir = os.path.join(notebook_dir, '../data_processing')  # Fichiers sources
data_fin_dir = os.path.join(notebook_dir, '../data_fin')  # Fichiers nettoyés finaux

# Charger la base fusionnée déjà définie (merged_df est supposé existant)
output_cleaned_merged = os.path.join(data_fin_dir, 'tweets_fusionnes.xlsx')

# Nettoyer et sauvegarder la base fusionnée
clean_tweets(
    merged_df,
    output_cleaned=output_cleaned_merged
)

# Charger la base bluesky_nettoye depuis data_procesing
df2_path = os.path.join(data_processing_dir, 'bluesky_nettoye.xlsx')
df2 = pd.read_excel(df2_path)

# Nettoyer et sauvegarder la base bluesky_nettoye
output_cleaned_df2 = os.path.join(data_fin_dir, 'bluesky_nettoye.xlsx')

clean_tweets(
    df2,
    output_cleaned=output_cleaned_df2
)


Total de contenus uniques : 12862
Nombre de contenus répétés (>= 4 fois) : 40
Nombre de tweets à supprimer : 237
237 tweets supprimés
Total de contenus uniques : 78939
Nombre de contenus répétés (>= 4 fois) : 145
Nombre de tweets à supprimer : 821
821 tweets supprimés


## 4. Apparence de la base de données

In [12]:
# Modifier les paramètres d'affichage de pandas
pd.set_option('display.max_colwidth', None)  # Affiche tout le contenu des colonnes
pd.set_option('display.max_columns', None)  # Affiche toutes les colonnes
pd.set_option('display.width', 1000)  # Ajuste la largeur totale de l'affichage

# Charger le fichier Excel
file_path = os.path.join(data_fin_dir, 'tweets_fusionnes.xlsx')
df = pd.read_excel(file_path)

# Afficher les 5 premières lignes
display(df.head(5))


,Username,Date,Content,Comments,Repost,Likes,Views,ConvertedDate,YearWeek,YearMonth
0,𝐇𝐎𝐓𝐄𝐏 悪いバナー,Nov 30,"The left have become so dependent on Twitter that they're forced with two options:\n\n1) Leave Twitter and leave behind the one platform that they feel grants them so much power.\n\n2) Stay on Twitter but deal with Musk, MAGA, Trump, etc.",1,0,0,49,2024-11-30,2024-47,2024-11
1,Gordon Gartrelle,Nov 30,Wow. This might be the post that gets me to quit Twitter. Musk is out of control.,0,0,1,9,2024-11-30,2024-47,2024-11
2,MAGA Destroyed America,Nov 30,Most are leaving Twitter because they no longer want to support the Elon Musk/MAGA engine.,0,0,1,7,2024-11-30,2024-47,2024-11
3,MercadoMagico.com USA,Nov 30,Change to Twitter Suggests Elon Musk Is Panicking Over Users Leaving for #Bluesky,0,2,0,43,2024-11-30,2024-47,2024-11
4,Maliah Well known (Gone/Left),Nov 30,"I am Leaving Twitter Forever. But I'll be on Bluesky, it's way better than twitter. So to my friends or anyone who's following me, I'll only be active on Bluesky. I will never support Donald Trump, I despise that Fascist including Elon Musk. Goodbye Forever ""Twitter"". Hi Bluesky",0,0,0,39,2024-11-30,2024-47,2024-11
